### Test loading Magma results

In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
basedir = 'C:\\Users\\rpetr\\OneDrive\\Desktop\\DISS_CODE\\ms2ldaviz\\ms2ldaviz'
sys.path.append(basedir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import django
django.setup()
from django.db import transaction

from basicviz.models import Experiment, Mass2Motif, Document, FeatureInstance, MagmaSub, FeatureInstance2Sub

MEDIA_ROOT is C:\Users\rpetr\OneDrive\Desktop\DISS_CODE\ms2ldaviz\ms2ldaviz\media


In [8]:
experiment = 'massbank_binned_005'
e = Experiment.objects.get(name = experiment)

In [9]:
documents = Document.objects.filter(experiment=e)

In [10]:
len(documents)

2132

In [11]:
import json

In [12]:
fname = '/home/joewandy/Documents/massbank_doc_annotation.json'
with open(fname) as f:
    data = json.load(f)

IOError: [Errno 2] No such file or directory: '/home/joewandy/Documents/massbank_doc_annotation.json'

In [13]:
with transaction.atomic():
    i = 0
    feature_map = {}
    for d in data:
        # update mol string in a document
        name = d['name']
        print '%d/%d %s' % (i, len(data), name)
        document = Document.objects.get(experiment=e, name=name)
        document.mol_string = d['mol']
        document.save()
        
        # lookup for existing feature instances in this document
        feature_instances = FeatureInstance.objects.filter(document=document)
        for f in feature_instances:
            feature_map[f.feature.name] = f
            
        # create a new substructure object
        for magma_annot in d['features']:
            name = magma_annot['name']
            feature = feature_map[name]
            for m in magma_annot['matches']:
                smiles = m['smiles']
                mol = m['mol'] if 'mol' in m else None
                fragatoms = m['fragatoms']
                mz = m['mz']
                sub_type = m['type'] if 'type' in m else None
                sub, _ = MagmaSub.objects.get_or_create(smiles=smiles, mol_string=mol)
                f2sub = FeatureInstance2Sub.objects.get_or_create(
                    feature=feature, sub=sub, fragatoms=fragatoms,
                    mz=mz, sub_type=sub_type
                )
        
        i+=1
        if i > 10:
            break

NameError: name 'data' is not defined